# Explore Data


In [2]:
import glob
import pandas as pd

SILVER_QA_PATH = '/lakehouse/default/Files/silver/igra2/qa'

In [12]:
# Get CSV files list from a folder
csv_files = glob.glob(SILVER_QA_PATH + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Pandas doesn't like to concatinate "empty" DataFrames
df_list = [dataframe for dataframe in df_list if dataframe.shape[0] > 0]

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True)

del df_list

In [13]:
print(f"Total Observation Count: {df.shape[0]}")

Total Observation Count: 346607


In [1]:
%%sql
select sl.id, sl.name, sl.state, count(*) as station_rows
from igra2_s_data_por_qa qa
inner join igra2_s_station_list sl
    on qa.id = sl.id
group by sl.id, sl.name, sl.state
order by 3 desc;

UsageError: Cell magic `%%sql` not found.


In [ ]:
%%sql
select hour, count(*) as obs_count, round(count(*) / (select count(*) from igra2_s_data_por_qa) * 100, 1) as pct
from igra2_s_data_por_qa
group by hour
order by 2 desc, 1

In [ ]:
%%sql
select
    l.name
    ,round(avg(usable_10k), 1) as avg_usable_10k
    ,round(avg(usable_10k) - std(usable_10k), 1) as usable_10k_68
    ,round(avg(usable_10k) - 2.0 * std(usable_10k), 1) as usable_10k_95
from igra2_s_data_por_qa qa
inner join igra2_s_station_list l
    on qa.id = l.id
where hour = 12
group by l.name
order by avg_usable_10k desc

In [ ]:
%%sql
select
    s.name
    ,count(*) as total_count
    ,sum(if(usable_10k < 20, 1, 0)) as thrown_out_count
    ,round(sum(if(usable_10k < 20, 1, 0)) / count(*) * 100.0, 1) as pct
from igra2_s_data_por_qa qa
inner join igra2_s_station_list s
    on s.id = qa.id
where hour = 12
group by s.name
order by pct desc;

In [ ]:
%%sql
select
    s.name as label
    ,min(usable_10k) as whislo
    ,percentile(usable_10k, 0.25) as q1
    ,percentile(usable_10k, 0.5) as med
    ,percentile(usable_10k, 0.75) as q3
    ,max(usable_10k) as whishi
from igra2_s_data_por_qa qa
inner join igra2_s_station_list s on qa.id = s.id
where hour = 12
group by s.name
order by med

In [ ]:
df = spark.sql("""select
    s.name as label
    ,min(usable_10k) as whislo
    ,percentile(usable_10k, 0.25) as q1
    ,percentile(usable_10k, 0.5) as med
    ,percentile(usable_10k, 0.75) as q3
    ,max(usable_10k) as whishi
from igra2_s_data_por_qa qa
inner join igra2_s_station_list s on qa.id = s.id
where hour = 12
group by s.name
order by med""").toPandas()

In [ ]:
boxes = []

for index, row in df.iterrows():
    boxes.append({
        'label' : row.label,
        'whislo': row.whislo,   # Bottom whisker position
        'q1'    : row.q1,       # First quartile (25th percentile)
        'med'   : row.med,      # Median         (50th percentile)
        'q3'    : row.q3,       # Third quartile (75th percentile)
        'whishi': row.whishi,   # Top whisker position
        'fliers': []            # Outliers
    })

fig, ax = plt.subplots()
ax.bxp(boxes, showfliers=False)
ax.set_ylabel("usable_10k")
plt.xticks(rotation=90)

In [ ]:
%%sql
select
    s.name
    ,round(avg(max_gph) / 1000.0, 1) as avg_max_gph
    ,round((avg(max_gph) - std(max_gph)) / 1000.0, 1) as gph_68
    ,round((avg(max_gph) - 2.0 * std(max_gph)) / 1000.0, 1) as gph_95
from igra2_s_data_por_qa qa
inner join igra2_s_station_list s on s.id = qa.id
group by s.name
order by avg_max_gph desc